<img src="https://raw.githubusercontent.com/seismo-live/seismo_live/refs/heads/master/notebooks/Workshops/2017_Baku_STCU_IRIS_ObsPy_course/images/obspy_logo_full_524x179px.png" width=90%> 

# Station Metainformation

In [ ]:
try:
  import obspy
except ModuleNotFoundError:
  !pip -qq install obspy
  import obspy

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 12, 8

- for station metadata, the de-facto standard of the future (replacing SEED/RESP) is [FDSN StationXML](http://www.fdsn.org/xml/station/)
- FDSN StationXML files can be read using **`read_inventory()`**

In [ ]:
!wget https://github.com/yannikbehr/BMKG_ObsPy_workshop/raw/refs/heads/main/data/indonesia_waveforms.tar.gz
!tar -xzf indonesia_waveforms.tar.gz

In [ ]:
from obspy import read_inventory
# real-world StationXML files often deviate from the official schema definition
# therefore file-format autodiscovery sometimes fails and we have to force the file format
inventory = read_inventory("./data/station_BBJI.xml", format="STATIONXML")
print(type(inventory))

- the nested ObsPy Inventory class structure (Inventory/Station/Channel/Response/...) is closely modelled after FDSN StationXML
<img src="https://raw.githubusercontent.com/seismo-live/seismo_live/refs/heads/master/notebooks/Workshops/2017_Baku_STCU_IRIS_ObsPy_course/images/Inventory.svg" width=90%>

In [ ]:
!head data/station_BBJI.xml

In [ ]:
print(inventory)

In [ ]:
network = inventory[0]
print(network)

In [ ]:
station = network[0]
print(station)

In [ ]:
channel = station[0]
print(channel)

In [ ]:
print(channel.response)

In [ ]:
from obspy import read
st = read("./data/GE.BBJI..LHZ.mseed")
print(st)

In [ ]:
inv = read_inventory("./data/station_BBJI.xml", format="STATIONXML")

In [ ]:
print(st[0].stats)

- the instrument response can be deconvolved from the waveform data using the convenience method **`Stream.remove_response()`**
- evalresp is used internally to calculate the instrument response

In [ ]:
st.plot()
st.remove_response(inventory=inv)
st.plot();

- several options can be used to specify details of the deconvolution (water level, frequency domain prefiltering), output units (velocity/displacement/acceleration), demeaning, tapering and to specify if any response stages should be omitted

In [ ]:
st = read("./data/GE.BBJI..LHZ.mseed")
st.remove_response(inventory=inv, water_level=60, pre_filt=(0.01, 0.02, 8, 10), output="DISP")
st.plot();

- station metadata not present in StationXML yet but in Dataless SEED or RESP files can be used for instrument correction using the `.simulate()` method of Stream/Trace in a similar fashion